In [7]:
#推論 OpenAI SDK
import os
from openai import OpenAI

base_url = os.getenv("OPENAI_BASE_URL", "http://ollama-runtime:11434/v1")
api_key  = os.getenv("OPENAI_API_KEY", "ollama")
model_id = os.getenv("OLLAMA_MODEL", "gpt-oss:120b")

# OpenAIの書式まま、Ollamaサーバーへリクエストを送るための同期クライアント
client = OpenAI(
    base_url=base_url,
    api_key=api_key,
)

# モデルの確認
try:
    models = [m.id for m in client.models.list().data]
    print("models:", models)
except Exception as e:
    print("models list failed:", e)
    models = []

# 推論
chat = client.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "system", "content": "あなたは日本語で回答する優秀なアシスタントです"},
        {"role": "user", "content": "鬼滅の刃のおすすめのキャラクターは？"},
    ],
)
print(chat.choices[0].message.content)

models: ['gpt-oss:120b']
もちろんです！『鬼滅の刃』は個性豊かなキャラクターがたくさん登場するので、好きな要素や好みで選ぶとより楽しめます。以下は特におすすめのキャラクターと、その魅力ポイントをご紹介します。

## 1. 炭治郎（たんじろう）  
**魅力ポイント**  
- **優しさと強い意志**：家族を守るために鬼と戦う姿勢が感動的です。  
- **成長ストーリー**：初心者から最強の剣士へと段階的に成長する過程が丁寧に描かれています。  
- **呼吸法**：水の呼吸と後に習得する日輪の呼吸の両方が見どころです。

## 2. 禰豆子（ねずこ）  
**魅力ポイント**  
- **強さと可愛さの両立**：鬼になっても兄への愛情を失わず、戦闘でも圧倒的な回復力を発揮。  
- **変身の多様性**：血鬼術「血痕」が多彩で、戦況に応じて形態を変えるシーンが魅力的です。  
- **感情表現**：言葉がなくても眼差しや動作だけで感情が伝わる演出が秀逸です。

## 3. 善逸（ぜんいつ）  
**魅力ポイント**  
- **怯えながらも驚異的な実力**：恐がりだけど、眠ると圧倒的な速さと強さを見せる「雷の呼吸・壱ノ型」。  
- **コメディ要素**：ツッコミどころ満載の性格が作品全体にユーモアを添えます。  
- **成長**：自信を持ち始める過程が感情移入しやすいです。

## 4. 伊之助（いのすけ）  
**魅力ポイント**  
- **野性的な直感**：獣のような感覚と戦闘感覚がユニーク。  
- **自作武器「獣の呼吸」**：独特な剣技と身体能力が見どころ。  
- **仲間思い**：表向きは荒々しいが、仲間への思いやりが垣間見える瞬間があります。

## 5. 胡蝶しのぶ（こちょうしのぶ）  
**魅力ポイント**  
- **知性と策略**：薬学と毒の知識を駆使し、鬼を弱らせる戦い方が斬新です。  
- **優雅さと残酷さ**：蝶のような美しさと、実は非常に危険な戦闘スタイルのギャップが魅力。  
- **「蟲の呼吸」**：独自の呼吸法で独特の剣術を披露します。

## 6. 煉獄杏寿郎（れんごくきょうじゅろう）  
**魅力ポイント**  
- **熱血漢**：炎の呼吸と同じく熱い情熱で仲間を鼓舞します。  
- 

In [8]:
#ツール呼び出し
import os, shlex, subprocess
from openai import AsyncOpenAI
from agents import Agent, Runner, function_tool, OpenAIChatCompletionsModel, set_tracing_disabled

set_tracing_disabled(True)

base_url = os.getenv("OPENAI_BASE_URL", "http://ollama-runtime:11434/v1")
api_key  = os.getenv("OPENAI_API_KEY", "ollama")
model_id = os.getenv("OLLAMA_MODEL", "gpt-oss:120b")

# OpenAIの書式まま、Ollamaサーバーへリクエストを送るための非同期クライアント
client = AsyncOpenAI(base_url=base_url, api_key=api_key)

# 呼び出すツール
@function_tool
def sh(cmd: str) -> str:
    """安全な読み取り専用コマンドだけ許可して実行し、標準出力を返す。"""
    allowed = {"cat", "ls", "echo", "pwd", "head", "tail"}
    prog = shlex.split(cmd)[0] if cmd.strip() else ""
    if prog not in allowed:
        return f"拒否: 許可コマンド({', '.join(sorted(allowed))})のみ実行可"
    cp = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    out = cp.stdout.strip() or cp.stderr.strip() or "(no output)"
    return out[:8192]

# エージェント構築
agent = Agent(
    name="Assistant",
    instructions=(
        "あなたは日本語で回答します。必要なら `sh` ツールだけを用い、"
        "実行前に短く日本語でコマンドの目的を説明し、危険な操作は拒否してください。"
    ),
    model=OpenAIChatCompletionsModel(model=model_id, openai_client=client),
    tools=[sh],
)

# 実行
result = await Runner.run(agent, "カレントディレクトリの内容を一覧を表示してください。")
print(result.final_output)

現在のディレクトリの内容は以下の通りです。

```
total 32
drwxrwxr-x 5 ubuntu ubuntu 4096 Sep 22 11:39 .
drwxr-xr-x 1 root   root     51 Sep 22 06:41 ..
drwx------ 4 root   root   4096 Aug 14 07:24 .Trash-0
-rw-rw-r-- 1 ubuntu ubuntu  300 Sep 22 11:21 .env
drwxr-xr-x 2 root   root   4096 Sep 22 11:32 .ipynb_checkpoints
lrwxrwxrwx 1 root   root      9 Sep 22 06:41 .venv -> /app/venv
-rw-rw-r-- 1 ubuntu ubuntu 1465 Sep 22 04:59 Dockerfile
-rw-rw-r-- 1 ubuntu ubuntu 1611 Sep 22 05:20 docker-compose.yml
-rw-r--r-- 1 ubuntu ubuntu  729 Sep 22 07:35 pyproject.toml
drwxr-xr-x 5 root   root   4096 Sep 22 11:53 src
```


In [9]:
#推論 Langchain Ollama
import os
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage

base_url = os.getenv("OPENAI_BASE_URL", "http://ollama-runtime:11434")
base_url = base_url[:-3] if base_url.endswith("/v1") else base_url
model_id = os.getenv("OLLAMA_MODEL", "gpt-oss:120b")

#ChatOllamaのクライアントを初期化
llm = ChatOllama(model=model_id, base_url=base_url, temperature=0.1)

#回答を生成
resp = llm.invoke([
    SystemMessage(content="日本語で丁寧に答えてください"),
    HumanMessage(content="鬼滅の刃のおすすめキャラは？"),
])

print(getattr(resp, "content", str(resp)))

はい、承知いたしました。『鬼滅の刃』は個性豊かなキャラクターが多数登場しますので、いくつかおすすめのキャラをご紹介いたします。ご自身の好みや視聴目的に合わせて、ぜひ参考にしてみてください。

---

## 1. 炭治郎（たんじろう）  
**理由**  
- 主人公としての成長が丁寧に描かれ、感情移入しやすい。  
- 「水の呼吸」や後に習得する「日の呼吸」など、剣技の変遷が見どころ。  
- 家族への思いやりや仲間への優しさが、物語全体のテーマを象徴しています。

## 2. 禰豆子（ねずこ）  
**理由**  
- 炭治郎の妹でありながら、人間と鬼の狭間に生きる独特の存在感。  
- 戦闘時の「血鬼術」や、感情表現が豊かで、視覚的にも魅力的です。  
- 「妹を守る」という強い意志が、作品の感動ポイントの一つです。

## 3. 煉獄（れんごく）杏寿郎（きょうじゅろう）  
**理由**  
- 炎の呼吸の使い手で、熱血漢かつ正義感が強いキャラクター。  
- 「炎のように燃える」姿勢が、観る者に勇気と感動を与えます。  
- 物語の重要な転換点で大きな活躍を見せ、ファンからの支持が高いです。

## 4. 胡蝶（こちょう）しのぶ  
**理由**  
- 「虫の呼吸」の使い手で、戦闘スタイルが独特かつ美しい。  
- 医療知識と薬学に長けており、戦闘だけでなく「癒し」の側面も持ちます。  
- 表向きは穏やかですが、内に秘めた強さと覚悟が魅力的です。

## 5. 宇髄（うすい）天元（てんげん）  
**理由**  
- 「音の呼吸」の使い手で、派手な演出と軽快な戦闘が特徴。  
- 明るく陽気な性格ながら、実力はトップクラス。  
- コミカルなシーンとシリアスなシーンのバランスが取れており、作品に彩りを添えます。

## 6. 伊之助（いのすけ）  
**理由**  
- 「獣の呼吸」の使い手で、野性的な戦闘スタイルが新鮮。  
- 直感的で率直な性格が、時に笑いを、時に熱い友情を生み出します。  
- 成長過程で見せる人間味が、視聴者の共感を呼びます。

## 7. 甘露寺（かんろじ）蜜璃（みつり）  
**理由**  
- 「恋の呼吸」の使い手で、柔らかく優しい雰囲気が魅力。  
- 圧倒的な身体能力と、仲間への思いやりが印象的です。  
- 彼女